In [ ]:
#!pip install graphdatascience

In [1]:
from graphdatascience import GraphDataScience
import pandas as pd

# ICPSRSKG_EG
host = 'bolt://localhost:7687'
user = 'neo4j'
password = "ICPSRCHATEXP"

gds = GraphDataScience(host, auth=(user, password))

### Import datasets

In [7]:
import json
PATH_data_export = "/Users/lizhouf/Documents/4_MICA/Scholarly Object Management + RecSys/DataChat/Data_result/"

with open(PATH_data_export+"data_output_20_22.json") as file:
    data_output = json.load(file)

In [8]:
data_output[:3]

[{'id': 34369,
  'name': 'The Iowa Adoption Studies, 1975-2008',
  'date': '2020-11-09',
  'url': 'https://doi.org/10.3886/ICPSR34369.v1 ',
  'series': 'NoRecord',
  'owner': 'NAHDAP',
  'users': 0,
  'dataUsers': 0,
  'dataReferences': 0,
  'funders': [{'name': 'NoRecord'}],
  'locations': [{'name': 'Iowa'}, {'name': ' United States'}],
  'terms': [{'name': 'adoption'},
   {'name': ' alcohol abuse'},
   {'name': ' alcoholism'},
   {'name': ' anxiety'},
   {'name': ' birth records'},
   {'name': ' cognitive functioning'},
   {'name': ' drug abuse'},
   {'name': ' environment'},
   {'name': ' genetics'},
   {'name': ' health problems'},
   {'name': ' medical history'},
   {'name': ' parent child relationship'},
   {'name': ' parental attitudes'},
   {'name': ' personality disorders'},
   {'name': ' post-traumatic stress disorder'},
   {'name': ' psychological evaluation'},
   {'name': ' substance abuse'},
   {'name': ' substance abuse treatment'},
   {'name': ' suicide'},
   {'name': ' 

In [9]:
gds.run_cypher(
    """
CREATE CONSTRAINT IF NOT EXISTS FOR (a:Dataset) REQUIRE a.id IS UNIQUE;
"""
)
gds.run_cypher(
    """
CREATE TEXT INDEX datasetname IF NOT EXISTS FOR (a:Dataset) ON a.name;
"""
)
gds.run_cypher(
    """
CREATE CONSTRAINT IF NOT EXISTS FOR (s:Series) REQUIRE s.name IS UNIQUE;
"""
)
gds.run_cypher(
    """
CREATE CONSTRAINT IF NOT EXISTS FOR (o:Owner) REQUIRE o.name IS UNIQUE;
"""
)
gds.run_cypher(
    """
CREATE CONSTRAINT IF NOT EXISTS FOR (f:Funder) REQUIRE f.name IS UNIQUE;
"""
)
gds.run_cypher(
    """
CREATE CONSTRAINT IF NOT EXISTS FOR (l:Location) REQUIRE l.name IS UNIQUE;
"""
)
gds.run_cypher(
    """
CREATE CONSTRAINT IF NOT EXISTS FOR (t:Term) REQUIRE t.name IS UNIQUE;
"""
)
gds.run_cypher(
    """
CREATE CONSTRAINT IF NOT EXISTS FOR (p:Publication) REQUIRE p.name IS UNIQUE;
"""
)

""


In [ ]:
# may not use
# SET a += apoc.map.clean(row, ["id", "name", "date", "url", "totalUserCount", "dataUserCount", "dataRefCount"], [])


In [10]:
dataset_import_query = """
UNWIND $data AS row
MERGE (a:Dataset {id: row.id})
SET a.name = row.name
SET a.date = datetime(replace(row.date, " ", "T"))
SET a.url = row.url
SET a.totalUserCount = row.users
SET a.dataUserCount = row.dataUsers
SET a.dataRefCount = row.dataReferences

WITH a, row, row.series AS series
MERGE (s:Series {name: series})
MERGE (a)-[:HAS_SERIES]->(s)

WITH a, row, row.owner AS owner
MERGE (o:Owner {name: owner})
MERGE (a)-[:HAS_OWNER]->(o)

FOREACH (entity in row.funders | 
  MERGE (f:Funder {name: entity.name})
  MERGE (a)-[:HAS_FUNDER]->(f)
)

FOREACH (entity in row.locations | 
  MERGE (l:Location {name: entity.name})
  MERGE (a)-[:HAS_LOCATION]->(l)
)

FOREACH (entity in row.terms | 
  MERGE (t:Term {name: entity.name})
  MERGE (a)-[:HAS_TERM]->(t)
)

FOREACH (entity in row.publications | 
  MERGE (p:Publication {name: entity.name, pubRefCount: entity.pubRefCount})
  MERGE (a)-[:CITED_BY]->(p)
)
"""
# TKTK add internal links for terms, and locations
#, type: entity.type
gds.run_cypher(dataset_import_query, {"data": data_output})

# later can add types, e.g. funder type, location type (country, etc.)

""
